# Treinar RL para equilibrar o Cartpole

Este notebook faz parte do [Currículo de IA para Iniciantes](http://aka.ms/ai-beginners). Foi inspirado pelo [tutorial oficial do PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) e por [esta implementação de Cartpole em PyTorch](https://github.com/yc930401/Actor-Critic-pytorch).

Neste exemplo, usaremos RL para treinar um modelo a equilibrar uma vara em um carrinho que pode se mover para a esquerda e para a direita em uma escala horizontal. Utilizaremos o ambiente [OpenAI Gym](https://www.gymlibrary.ml/) para simular a vara.

> **Nota**: Pode executar o código desta lição localmente (por exemplo, no Visual Studio Code), caso em que a simulação será aberta numa nova janela. Ao executar o código online, pode ser necessário fazer alguns ajustes no código, conforme descrito [aqui](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Começaremos por garantir que o Gym está instalado:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Agora vamos criar o ambiente CartPole e ver como operá-lo. Um ambiente tem as seguintes propriedades:

* **Action space** é o conjunto de ações possíveis que podemos realizar em cada passo da simulação  
* **Observation space** é o espaço de observações que podemos fazer  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Vamos ver como a simulação funciona. O seguinte ciclo executa a simulação até que `env.step` não devolva o sinal de terminação `done`. Vamos escolher ações aleatoriamente usando `env.action_space.sample()`, o que significa que a experiência provavelmente falhará muito rapidamente (o ambiente CartPole termina quando a velocidade do CartPole, a sua posição ou o ângulo estão fora de certos limites).

> A simulação será aberta numa nova janela. Pode executar o código várias vezes e observar como se comporta.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Pode notar que as observações contêm 4 números. São eles:
- Posição do carrinho
- Velocidade do carrinho
- Ângulo da barra
- Taxa de rotação da barra

`rew` é a recompensa que recebemos em cada passo. Pode ver que, no ambiente CartPole, recebe-se 1 ponto por cada passo de simulação, e o objetivo é maximizar a recompensa total, ou seja, o tempo que o CartPole consegue equilibrar sem cair.

Durante o aprendizado por reforço, o nosso objetivo é treinar uma **política** $\pi$, que para cada estado $s$ nos dirá qual ação $a$ tomar, essencialmente $a = \pi(s)$.

Se quiser uma solução probabilística, pode pensar na política como retornando um conjunto de probabilidades para cada ação, ou seja, $\pi(a|s)$ significaria a probabilidade de que devemos tomar a ação $a$ no estado $s$.

## Método de Gradiente de Política

No algoritmo mais simples de RL, chamado **Gradiente de Política**, iremos treinar uma rede neural para prever a próxima ação.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Vamos treinar a rede realizando muitos experimentos e atualizando a nossa rede após cada execução. Vamos definir uma função que irá executar o experimento e retornar os resultados (o chamado **rastro**) - todos os estados, ações (e as suas probabilidades recomendadas) e recompensas:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Pode executar um episódio com uma rede não treinada e observar que a recompensa total (ou seja, a duração do episódio) é muito baixa:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Um dos aspetos complicados do algoritmo de gradiente de política é usar **recompensas descontadas**. A ideia é que calculamos o vetor de recompensas totais em cada etapa do jogo e, durante este processo, descontamos as recompensas iniciais usando algum coeficiente $gamma$. Também normalizamos o vetor resultante, porque o usaremos como peso para afetar o nosso treino:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Agora vamos começar o treino! Iremos executar 300 episódios, e em cada episódio faremos o seguinte:

1. Executar o experimento e recolher o traço.
1. Calcular a diferença (`gradients`) entre as ações realizadas e as probabilidades previstas. Quanto menor for a diferença, mais certeza teremos de que tomámos a ação correta.
1. Calcular recompensas descontadas e multiplicar os gradientes pelas recompensas descontadas - isso garantirá que os passos com recompensas mais altas terão maior impacto no resultado final do que aqueles com recompensas mais baixas.
1. As ações-alvo esperadas para a nossa rede neural serão parcialmente derivadas das probabilidades previstas durante a execução e parcialmente dos gradientes calculados. Utilizaremos o parâmetro `alpha` para determinar em que medida os gradientes e as recompensas são considerados - isto é chamado de *taxa de aprendizagem* do algoritmo de reforço.
1. Por fim, treinamos a nossa rede com os estados e ações esperadas, e repetimos o processo.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Agora vamos executar o episódio com renderização para ver o resultado:


In [ ]:
_ = run_episode(render=True)

Esperemos que consiga ver que o bastão agora consegue equilibrar-se bastante bem!

## Modelo Actor-Critic

O modelo Actor-Critic é um desenvolvimento adicional dos gradientes de política, no qual construímos uma rede neural para aprender tanto a política como as recompensas estimadas. A rede terá dois outputs (ou pode ser vista como duas redes separadas):
* **Actor** irá recomendar a ação a tomar, fornecendo-nos a distribuição de probabilidade do estado, como no modelo de gradiente de política.
* **Critic** estimará qual seria a recompensa dessas ações. Retorna as recompensas totais estimadas no futuro para o estado dado.

Vamos definir um modelo deste tipo:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Precisaríamos modificar ligeiramente as nossas funções `discounted_rewards` e `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Agora vamos executar o ciclo principal de treino. Utilizaremos o processo manual de treino da rede, calculando as funções de perda adequadas e atualizando os parâmetros da rede:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Conclusão

Vimos dois algoritmos de RL nesta demonstração: o simples policy gradient e o mais sofisticado actor-critic. Pode-se observar que esses algoritmos operam com noções abstratas de estado, ação e recompensa - o que significa que podem ser aplicados a ambientes muito diferentes.

O reinforcement learning permite-nos aprender a melhor estratégia para resolver um problema apenas analisando a recompensa final. O facto de não precisarmos de conjuntos de dados rotulados permite-nos repetir simulações várias vezes para otimizar os nossos modelos. No entanto, ainda existem muitos desafios no RL, que poderá explorar caso decida aprofundar-se mais nesta área fascinante da IA.



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante notar que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes da utilização desta tradução.
